In [100]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import string
import re
import numpy as np
# download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
df = pd.read_csv('articles1.csv')
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [ ]:
#selecting only the first 100 rows
df= df.iloc[:100]

from google.colab import files
# saving the 100 articles to a csv file
df.to_csv("100articles.csv")
#downloading the csv file 
files.download("100articles.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [105]:
df = pd.read_csv('100articles.csv', index_col= [0])
df.head()

,Unnamed: 0.1,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [106]:
# calculate the length of each string in the column
lengths = [len(str(s)) for s in df['content']]

# calculate the average length of the strings
average_length = sum(lengths) / len(lengths)

# print the result
print("The average text length in the column is:", average_length)

The average text length in the column is: 7438.79


In [107]:
#creating a new dataframe with just the column  content
df2 = pd.DataFrame({'articles': df['content']})
df2.info()

df2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   articles  100 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


,articles
0,WASHINGTON — Congressional Republicans have...
1,"After the bullet shells get counted, the blood..."
2,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Death may be the great equalizer, but it isn’t..."
4,"SEOUL, South Korea — North Korea’s leader, ..."


In [108]:
# calculate the length of each string in the column
lengths = [len(str(s)) for s in df2['articles']]

# calculate the average length of the strings
average_length = sum(lengths) / len(lengths)

# print the result
print("The average text length in the column is:", average_length)

The average text length in the column is: 7438.79


In [109]:
#to print out one of the articles
#we will use this to see the change in texts
print(df2.iloc[56, 0])

Alicia Rivera, a    single mother with four children and big dreams, must budget her money and her time wisely. “I got my kids,” Ms. Rivera said. “I don’t want to waste my time. ” Ms. Rivera and her children  —   Joaquin, 17 Nelson, 16 Alyssa, 11 and Rafael, 2  —   live in a   apartment in the Bronx, near the southern tip of the Bronx River Parkway. The past few years have sometimes been overwhelming for her as she tries to raise her children by herself, get an education and lay the foundation for a professional career. When she was seven months pregnant with Rafael, Ms. Rivera injured her back while working as a home health aide. Soon after he was born, she switched to another job, helping   people navigate complicated real estate and tax forms and other paperwork. Ms. Rivera makes about $100 every week. She also receives $216 a month in Supplemental Security Income and $506 a month in food stamps. With a monthly rent of $308, she needs to keep a frugal budget. But Ms. Rivera has a pl

# Preprocessing the texts

In [110]:
# define preprocessing function
def preprocess_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    # tokenize text into words
    tokens = word_tokenize(text.lower())
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
   
    # join tokens back into text
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

# apply preprocessing function to 'Text' column
df2['preprocessed articles'] = df2['articles'].apply(preprocess_text)

# print preprocessed texts
print(df2['preprocessed articles'])

0     washington congressional republicans new fear ...
1     bullet shells get counted blood dries votive c...
2     walt disney bambi opened 1942 critics praised ...
3     death may great equalizer necessarily evenhand...
4     seoul south korea north korea leader kim said ...
                            ...                        
95    five years investigations negotiations curious...
96    question analyst thursday delicate enough agre...
97    long island rail road train crashed brooklyn w...
98    detroit unexpectedly strong sales new vehicles...
99    struggling sagging sales another crucial holid...
Name: preprocessed articles, Length: 100, dtype: object


In [111]:
#to print out one of the articles
print(len(df2.iloc[56, 1]))

2877


# Function to Summarize the Texts

In [112]:
# Define the function to generate summaries using TextRank
def summarize(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    # Generate a matrix of sentence similarity scores
    sim_matrix = cosine_similarity(tfidf_vectors)
    # Build the sentence graph
    sentence_graph = nx.from_numpy_array(sim_matrix)
    # Calculate the TextRank scores for each sentence
    scores = nx.pagerank(sentence_graph)
    # Sort the sentences by their TextRank scores
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # Extract the top 3 sentences as the summary
    summary = ' '.join([s[1] for s in ranked_sentences[:1]])
    return summary

# apply preprocessing function to 'Text' column
df2['summarised articles'] = df2['preprocessed articles'].apply(preprocess_text)

# print preprocessed texts
print(df2['summarised articles'])

0     washington congressional republicans new fear ...
1     bullet shells get counted blood dries votive c...
2     walt disney bambi opened 1942 critics praised ...
3     death may great equalizer necessarily evenhand...
4     seoul south korea north korea leader kim said ...
                            ...                        
95    five years investigations negotiations curious...
96    question analyst thursday delicate enough agre...
97    long island rail road train crashed brooklyn w...
98    detroit unexpectedly strong sales new vehicles...
99    struggling sagging sales another crucial holid...
Name: summarised articles, Length: 100, dtype: object


In [113]:
#to print out one of the articles
print(len(df2.iloc[56, 2]))

2877


In [114]:
# calculate the length of each string in the column
lengths = [len(str(s)) for s in df2['summarised articles']]

# calculate the average length of the strings
average_length = sum(lengths) / len(lengths)

# print the result
print("The average text length in the column is:", average_length)

The average text length in the column is: 5080.1


In [115]:
# Define the function to generate summaries using TextRank
def summarize(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    # Generate a matrix of sentence similarity scores
    sim_matrix = cosine_similarity(tfidf_vectors)
    # Build the sentence graph
    sentence_graph = nx.from_numpy_array(sim_matrix)
    # Calculate the TextRank scores for each sentence
    scores = nx.pagerank(sentence_graph)
    # Sort the sentences by their TextRank scores
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    # Extract the top 30% of the sentences as the summary
    num_sentences = max(1, round(len(sentences) * 0.3))
    summary = ' '.join([s[1] for s in ranked_sentences[:num_sentences]])
    return summary

# apply preprocessing function to 'Text' column
df2['new sum2'] = df2['preprocessed articles'].apply(summarize)

# print preprocessed texts
print(df2['new sum2'])


0     washington congressional republicans new fear ...
1     bullet shells get counted blood dries votive c...
2     walt disney bambi opened 1942 critics praised ...
3     death may great equalizer necessarily evenhand...
4     seoul south korea north korea leader kim said ...
                            ...                        
95    five years investigations negotiations curious...
96    question analyst thursday delicate enough agre...
97    long island rail road train crashed brooklyn w...
98    detroit unexpectedly strong sales new vehicles...
99    struggling sagging sales another crucial holid...
Name: new sum2, Length: 100, dtype: object


In [85]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   articles               100 non-null    object
 1   preprocessed articles  100 non-null    object
 2   summarised articles    100 non-null    object
 3   new sum                100 non-null    object
 4   new sum2               100 non-null    object
dtypes: object(5)
memory usage: 8.8+ KB


In [116]:
# calculate the length of each string in the column
lengths = [len(str(s)) for s in df2['new sum2']]

# calculate the average length of the strings
average_length = sum(lengths) / len(lengths)

# print the result
print("The average text length in the column is:", average_length)

The average text length in the column is: 5080.1


In [96]:
#to print out one of the articles
print(len(df2.iloc[56, 4]))

2877


In [119]:
def get_sentence_similarity(sentences):
    # create a matrix of pairwise sentence similarity
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = cosine_similarity(sentences[i], sentences[j])[0][0]
    return similarity_matrix

def get_summary(article_text):
    # preprocess the article text
    processed_text = preprocess_text(article_text)
    
    # split the text into individual sentences
    sentences = sent_tokenize(processed_text)
    
    # get the similarity between sentences
    sentence_similarity_matrix = get_sentence_similarity(sentences)
    
    # apply the TextRank algorithm
    scores = nx.pagerank(nx.from_numpy_array(sentence_similarity_matrix))
    
    # sort the sentences by their scores
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    # generate the summary by selecting the top 3 sentences
    summary_sentences = ranked_sentences[:3]
    summary = ' '.join([s[1] for s in summary_sentences])
    
    return summary

#create a new column for the summary
df2['new sum3'] = df2['preprocessed articles'].apply(get_summary)

In [120]:
# calculate the length of each string in the column
lengths = [len(str(s)) for s in df2['new sum3']]

# calculate the average length of the strings
average_length = sum(lengths) / len(lengths)

# print the result
print("The average text length in the column is:", average_length)

The average text length in the column is: 5080.1


In [123]:
contractions_dict = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"doesn’t": "does not",
"don't": "do not",
"don’t": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y’all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"ain’t": "am not",
"aren’t": "are not",
"can’t": "cannot",
"can’t’ve": "cannot have",
"’cause": "because",
"could’ve": "could have",
"couldn’t": "could not",
"couldn’t’ve": "could not have",
"didn’t": "did not",
"doesn’t": "does not",
"don’t": "do not",
"don’t": "do not",
"hadn’t": "had not",
"hadn’t’ve": "had not have",
"hasn’t": "has not",
"haven’t": "have not",
"he’d": "he had",
"he’d’ve": "he would have",
"he’ll": "he will",
"he’ll’ve": "he will have",
"he’s": "he is",
"how’d": "how did",
"how’d’y": "how do you",
"how’ll": "how will",
"how’s": "how is",
"i’d": "i would",
"i’d’ve": "i would have",
"i’ll": "i will",
"i’ll’ve": "i will have",
"i’m": "i am",
"i’ve": "i have",
"isn’t": "is not",
"it’d": "it would",
"it’d’ve": "it would have",
"it’ll": "it will",
"it’ll’ve": "it will have",
"it’s": "it is",
"let’s": "let us",
"ma’am": "madam",
"mayn’t": "may not",
"might’ve": "might have",
"mightn’t": "might not",
"mightn’t’ve": "might not have",
"must’ve": "must have",
"mustn’t": "must not",
"mustn’t’ve": "must not have",
"needn’t": "need not",
"needn’t’ve": "need not have",
"o’clock": "of the clock",
"oughtn’t": "ought not",
"oughtn’t’ve": "ought not have",
"shan’t": "shall not",
"sha’n’t": "shall not",
"shan’t’ve": "shall not have",
"she’d": "she would",
"she’d’ve": "she would have",
"she’ll": "she will",
"she’ll’ve": "she will have",
"she’s": "she is",
"should’ve": "should have",
"shouldn’t": "should not",
"shouldn’t’ve": "should not have",
"so’ve": "so have",
"so’s": "so is",
"that’d": "that would",
"that’d’ve": "that would have",
"that’s": "that is",
"there’d": "there would",
"there’d’ve": "there would have",
"there’s": "there is",
"they’d": "they would",
"they’d’ve": "they would have",
"they’ll": "they will",
"they’ll’ve": "they will have",
"they’re": "they are",
"they’ve": "they have",
"to’ve": "to have",
"wasn’t": "was not",
"we’d": "we would",
"we’d’ve": "we would have",
"we’ll": "we will",
"we’ll’ve": "we will have",
"we’re": "we are",
"we’ve": "we have",
"weren’t": "were not",
"what’ll": "what will",
"what’ll’ve": "what will have",
"what’re": "what are",
"what’s": "what is",
"what’ve": "what have",
"when’s": "when is",
"when’ve": "when have",
"where’d": "where did",
"where’s": "where is",
"where’ve": "where have",
"who’ll": "who will",
"who’ll’ve": "who will have",
"who’s": "who is",
"who’ve": "who have",
"why’s": "why is",
"why’ve": "why have",
"will’ve": "will have",
"won’t": "will not",
"won’t’ve": "will not have",
"would’ve": "would have",
"wouldn’t": "would not",
"wouldn’t’ve": "would not have",
"y’all": "you all",
"y’all": "you all",
"y’all’d": "you all would",
"y’all’d’ve": "you all would have",
"y’all’re": "you all are",
"y’all’ve": "you all have",
"you’d": "you would",
"you’d’ve": "you would have",
"you’ll": "you will",
"you’ll’ve": "you will have",
"you’re": "you are",
"you’re": "you are",
"you’ve": "you have",
}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function expand the contractions if there's any
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [124]:
# Removing the contractions
df2['new sum4'] = df2['preprocessed articles'].apply(lambda x: expand_contractions(x))

In [132]:
# Function to calculate the word frequency
def word_frequency(article_word):
    word_frequency = {}
    li_word = []
    for sentence in article_word:
        for word in word_tokenize(sentence):
            if word not in word_frequency.keys():
                word_frequency[word] = 1
            else:
                word_frequency[word] += 1
        li_word.append(word_frequency)
        word_frequency = {}
    normalize(li_word)
    return normalized_freq

# Function to Score the sentence which is called in the function sent_token
def sentence_score(li):
    global sentence_score_list
    sentence_score = {}
    sentence_score_list = []
    for list_, dictionary in zip(li, normalized_freq):
        for sent in list_:
            for word in word_tokenize(sent):
                if word in dictionary.keys():
                    if sent not in sentence_score.keys():
                        sentence_score[sent] = dictionary[word]
                    else:
                        sentence_score[sent] += dictionary[word]
        sentence_score_list.append(sentence_score)
        sentence_score = {}
    return sentence_score_list

def summary(sentence_score_OwO):
    summary_list = []
    for summ in sentence_score_OwO:
        select_length = int(len(summ)*0.25)
        summary_ = nlargest(select_length, summ, key = summ.get)
        summary_list.append(".".join(summary_))
    return summary_list

def make_series(art):
    global dataframe
    data_dict = {'article' : [art]}
    dataframe = pd.DataFrame(data_dict)['article']
    return dataframe

def article_summarize(artefact):
    
    if type(artefact) != pd.Series:
        artefact = make_series(artefact)
    
    df = preprocessing(artefact)
    
    word_normalization = word_frequency(df)
    
    sentence_score_OwO = sent_token(article_sent)
    
    summarized_article = summary(sentence_score_OwO)
    
    return summarized_article



In [133]:
summaries = article_summarize(df2['new sum4'])
summaries

NameError: ignored